## Ford SoH EDA 

Création de ce notebokk car une décroissance élevé des soh des ford mach-e à été constatée.

In [ ]:
import pandas as pd
import numpy as np
import plotly.express as px
from core.sql_utils import *
from core.s3.s3_utils import S3Service
from core.s3.settings import S3Settings
from core.spark_utils import create_spark_session
from sqlalchemy import text
import pyspark.sql.functions as F

settings = S3Settings()

spark = create_spark_session(
    settings.S3_KEY,
    settings.S3_SECRET
)

s3 = S3Service()

In [ ]:
engine = get_sqlalchemy_engine()
con = engine.connect()

with engine.connect() as connection:
    dbeaver_df = pd.read_sql(text("""SELECT * FROM vehicle_data vd
            join vehicle v
            on v.id = vd.vehicle_id
            join vehicle_model vm 
            on vm.id = v.vehicle_model_id
            join battery b
            on b.id = vm.battery_id
            WHERE vm.oem_id = '437e8d1a-0200-41da-a06b-4400c4fa5720';"""), con)

In [ ]:
px.scatter(dbeaver_df, x='odometer', y='soh', color='vin')

In [ ]:
phase_results = s3.read_parquet_df_spark(spark, "result_phases/result_phases_ford.parquet")


In [ ]:
dbeaver_df.vin.unique()

In [ ]:
px.scatter(phase_results[phase_results['vin']=='WF0TK1EM5MMA31333'], x='ODOMETER_FIRST', y='SOH', color='VIN')

In [ ]:
px.scatter(phase_results[phase_results['vin']=='WF0TK1EM5MMA31333'], x='DATETIME_BEGIN', y='SOH', color='VIN')

In [ ]:
dbeaver_df.columns

In [ ]:
px.scatter(dbeaver_df, x='odometer', y='soh', color='vin')

There are 6 vin wich looks have a to low SoH

In [ ]:
vin_low = dbeaver_df[dbeaver_df["soh"]<0.8].vin.unique().tolist()

In [ ]:
vin_test = phase_results.filter(F.col("VIN").isin(vin_low))
px.scatter(vin_test, x='DATETIME_BEGIN', y='SOH', color='VIN')

In [ ]:
px.scatter(vin_test, x='ODOMETER_FIRST', y='SOH', color='VIN')

vin **WF0TK3SU5MMA16976** and **WF0TK1EM6MMA38274** don't look particulary weird, but the 4 other yes.   


In [ ]:
vin_soh_low = vin_test.toPandas()

In [ ]:
vin_soh_low.MODEL.unique(), vin_soh_low.BATTERY_NET_CAPACITY.unique()

In [ ]:
vin_soh_low.VIN.unique()

In [ ]:
vin_ford_low = ['WF0TK1EM2MMA16501', 'WF0TK1EM5MMA31333', 'WF0TK1EM2MMA16563', 'WF0TK1EM4MMA38533']
df_low_soh = vin_test.filter(F.col("VIN").isin(vin_ford_low)).toPandas()


In [ ]:
df_low_soh